In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
from ToolBox import CrossValidation
import json

In [2]:
train = pd.read_csv('./train.csv')
keys = ["OutcomeType", "AnimalType", "SexuponOutcome", "AgeuponOutcome", "Breed", "Color"]
train

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White


In [3]:
print "OutcomeType"

dict_temp = {}

dict_temp["OutcomeType"] = np.array([val for val in train.OutcomeType.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
dict_temp["OutcomeType"] = np.sort(dict_temp["OutcomeType"]) #To make easier the construction of submission.
dict_temp["OutcomeType"]

OutcomeType


array(['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'], 
      dtype='|S15')

In [4]:
#print "OutcomeSubtype"

#dict_temp["OutcomeSubtype"] = np.array([val for val in train.OutcomeSubtype.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
#dict_temp["OutcomeSubtype"] 

In [5]:
print "Breed"
dict_temp["Breed"] = np.array([val for val in train.Breed.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
np.set_printoptions(threshold='nan')
dict_temp["Breed"]

Breed


array(['Shetland Sheepdog Mix', 'Domestic Shorthair Mix', 'Pit Bull Mix',
       'Lhasa Apso/Miniature Poodle', 'Cairn Terrier/Chihuahua Shorthair',
       'American Pit Bull Terrier Mix', 'Cairn Terrier',
       'Miniature Schnauzer Mix', 'Yorkshire Terrier Mix',
       'Great Pyrenees Mix', 'Angora Mix', 'Flat Coat Retriever Mix',
       'Queensland Heeler Mix', 'Plott Hound/Boxer', 'German Shepherd Mix',
       'American Staffordshire Terrier Mix', 'Chihuahua Shorthair Mix',
       'Russian Blue Mix', 'Labrador Retriever Mix',
       'Collie Smooth/German Shepherd', 'Domestic Longhair Mix', 'Beagle',
       'Pit Bull/Labrador Retriever', 'Blue Lacy Mix',
       'Black Mouth Cur Mix', 'Dachshund/Beagle', 'Dachshund Mix',
       'Maltese Mix', 'Pointer Mix', 'English Foxhound', 'Vizsla Mix',
       'German Shepherd/Great Pyrenees',
       'Cardigan Welsh Corgi/Chihuahua Shorthair', 'Domestic Longhair',
       'Chow Chow/Pit Bull', 'Chihuahua Shorthair', 'Siamese Mix',
       'Domestic

In [6]:
print "AnimalType"
dict_temp["AnimalType"] = np.array([val for val in train.AnimalType.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
np.set_printoptions(threshold='nan')
dict_temp["AnimalType"]

AnimalType


array(['Dog', 'Cat'], 
      dtype='|S3')

In [7]:
print "SexuponOutcome"
dict_temp["SexuponOutcome"] = np.array([val for val in train.SexuponOutcome.unique() if str(val) != 'nan' and str(val).lower() != 'unknown'])
dict_temp["SexuponOutcome"]

SexuponOutcome


array(['Neutered Male', 'Spayed Female', 'Intact Male', 'Intact Female'], 
      dtype='|S13')

In [8]:
print "AgeuponOutcome"
dict_temp["AgeuponOutcome"] = train.AgeuponOutcome.unique()
dict_temp["AgeuponOutcome"] 

AgeuponOutcome


array(['1 year', '2 years', '3 weeks', '1 month', '5 months', '4 years',
       '3 months', '2 weeks', '2 months', '10 months', '6 months',
       '5 years', '7 years', '3 years', '4 months', '12 years', '9 years',
       '6 years', '1 weeks', '11 years', '4 weeks', '7 months', '8 years',
       '11 months', '4 days', '9 months', '8 months', '15 years',
       '10 years', '1 week', '0 years', '14 years', '3 days', '6 days',
       '5 days', '5 weeks', '2 days', '16 years', '1 day', '13 years', nan,
       '17 years', '18 years', '19 years', '20 years'], dtype=object)

In [9]:
print "Color"
dict_temp["Color"] = np.array([val for val in train.Color.unique() if str(val) != 'nan' and str(val).lower() != 'unknown'])
dict_temp["Color"]

dict_colors = {}

i = 1

for value in dict_temp["Color"]:
    if value not in dict_colors and '/' not in value:
        dict_colors[value] = i
        i+=1
    elif value not in dict_colors and  '/' in value:
        temp_val = value.split('/')
        temp = '{1}/{0}'.format(temp_val[0], temp_val[1]) 
        if temp not in dict_colors:
            dict_colors[value] = i
            i += 1

#print dict_colors
            
def getColorCode(colorValue):
    if colorValue in dict_colors:
        return dict_colors[colorValue]
    splits = colorValue.split('/')
    if len(splits) > 1: #Must be 2
        val = '{1}/{0}'.format(splits[0], splits[1])
        if val in dict_colors:
            return dict_colors[val]
    return 0

Color


In [10]:
dict_colors

{'Agouti': 142,
 'Agouti/Brown Tabby': 239,
 'Apricot': 72,
 'Apricot/Brown': 156,
 'Apricot/White': 57,
 'Black': 11,
 'Black Brindle': 96,
 'Black Brindle/Black': 154,
 'Black Brindle/Brown': 209,
 'Black Brindle/Brown Brindle': 262,
 'Black Brindle/White': 103,
 'Black Smoke': 90,
 'Black Smoke/Brown Tabby': 168,
 'Black Smoke/White': 85,
 'Black Tabby': 101,
 'Black Tabby/Orange': 155,
 'Black Tabby/White': 42,
 'Black Tiger/White': 250,
 'Black/Black': 144,
 'Black/Black Smoke': 165,
 'Black/Black Tabby': 186,
 'Black/Blue Merle': 123,
 'Black/Brown': 19,
 'Black/Brown Brindle': 44,
 'Black/Buff': 98,
 'Black/Gray': 53,
 'Black/Orange': 126,
 'Black/Red': 14,
 'Black/Silver Tabby': 130,
 'Black/Tan': 6,
 'Black/Tricolor': 71,
 'Black/White': 17,
 'Black/Yellow': 241,
 'Black/Yellow Brindle': 202,
 'Blue': 24,
 'Blue Cream': 4,
 'Blue Cream/Blue Tiger': 92,
 'Blue Cream/Buff': 223,
 'Blue Merle': 47,
 'Blue Merle/Blue Merle': 235,
 'Blue Merle/Brown': 105,
 'Blue Merle/Cream': 178,

In [11]:
import math

keys

['OutcomeType',
 'AnimalType',
 'SexuponOutcome',
 'AgeuponOutcome',
 'Breed',
 'Color']

In [12]:
def convertAgeToNum(age):
    if str(age).isdigit():
        return age
    elif str(age) == 'nan':
        return 0
    else:
        age = age.strip()
        age = age.lower()
        splits = age.split()
        if "year" in splits[1]:
            return float(splits[0]) * 365
        elif "month" in splits[1]:
            return float(splits[0]) * 30
        elif "week" in splits[1]:
            return float(splits[0]) * 7
        elif "day" in splits[1]:
            return float(splits[0])

In [13]:
train_transformed = []
for idx, row in train.iterrows():
    lis = []
    for key in keys:
        if key == "AgeuponOutcome":
            age_in_days = convertAgeToNum(row[key])
            lis.append(age_in_days)
        elif key == 'Color':
            if str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
                lis.append(getColorCode(row[key]))
            else:
                lis.append(0)
        elif str(row[key]) != "nan" and str(row[key]).lower() != "unknown":            
            temp = np.where(dict_temp[key] == row[key])[0][0]+1
            lis.append(temp)
        else:
            lis.append(0)
    train_transformed.append(np.array(lis))


In [14]:
train_transformed = np.array(train_transformed, np.int16)
print train_transformed

[[   4    1    1  365    1    1]
 [   3    2    2  365    2    2]
 [   1    1    1  730    3    3]
 [   5    2    3   21    2    4]
 [   5    1    1  730    4    5]
 [   5    1    4   30    5    6]
 [   5    2    3   21    2    7]
 [   5    2    0   21    2    8]
 [   1    1    2  150    6    9]
 [   1    1    2  365    7   10]
 [   5    2    0  730    2   11]
 [   1    1    2  730    8   12]
 [   1    1    1 1460    3   13]
 [   4    1    1  730    9   14]
 [   4    1    1  365   10   15]
 [   1    2    1   90    2   16]
 [   5    2    3   21    2    8]
 [   5    1    4   14    3    1]
 [   5    2    4  730   11   10]
 [   4    1    1  365   12   11]
 [   5    1    3   60   13   17]
 [   5    2    2  365    2   10]
 [   1    1    2  730   14   18]
 [   1    1    1  300   15    1]
 [   4    1    1   60   15   19]
 [   5    2    4   30    2   20]
 [   1    1    2  180   16   21]
 [   1    1    2 1825   17   22]
 [   1    2    2   60    2   23]
 [   4    2    1 2555   18   24]
 [   3    

In [15]:
#Train the model: RandomForest.

labels = [x[0] for x in train_transformed]
train = [x[1:] for x in train_transformed]

In [16]:
test = pd.read_csv('./test.csv')
test_keys = ["AnimalType", "SexuponOutcome", "AgeuponOutcome", "Breed", "Color"]
test

,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
2,3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
3,4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
4,5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White
5,6,Beau,2015-06-23 11:17:00,Dog,Neutered Male,3 years,Beagle Mix,Brown/White
6,7,Bobo,2014-03-12 09:45:00,Cat,Neutered Male,13 years,Domestic Medium Hair Mix,Brown Tabby/White
7,8,Abby,2014-06-25 08:27:00,Cat,Spayed Female,6 months,Domestic Shorthair Mix,Brown Tabby
8,9,Ruby Grace,2014-11-12 18:05:00,Dog,Spayed Female,3 months,Cairn Terrier,Black/Cream
9,10,Ruby,2014-04-07 17:41:00,Dog,Spayed Female,1 year,Pit Bull Mix,Brown/White


In [17]:
test_transformed = []
for idx, row in test.iterrows():
    lis = []
    for key in test_keys:
        if key == "AgeuponOutcome":
            age_in_days = convertAgeToNum(row[key])
            lis.append(age_in_days)
        elif key == 'Color':
            if str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
                lis.append(getColorCode(row[key]))
            else:
                lis.append(0)
        elif str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
            temp = np.where(dict_temp[key] == row[key])
            if len(temp[0]) > 0:
                lis.append(temp[0][0]+1)
            else:
                lis.append(0)
        else:
            lis.append(0)
    test_transformed.append(np.array(lis))

In [18]:
test_transformed = np.array(test_transformed, np.int16)
#np.set_printoptions(threshold='nan')
print test_transformed

[[   1    4  300   19    9]
 [   1    2  730  105    6]
 [   2    1  365    2    8]
 [   1    3  120  328   28]
 [   1    1  730   67   10]
 [   1    1 1095   48    1]
 [   2    1 4745   38   33]
 [   2    2  180    2    8]
 [   1    2   90    7  119]
 [   1    2  365    3    1]
 [   1    2  150   19   17]
 [   1    4  730    3    3]
 [   1    2  180    3   37]
 [   1    1  730   67   10]
 [   1    1  365  937   19]
 [   1    2  120  110   10]
 [   1    1 1825   72    6]
 [   1    1 1095  120   75]
 [   2    2 1095    2   26]
 [   2    4 1460    2   26]
 [   2    2  730    2   11]
 [   2    0   21    2   11]
 [   1    1  365  295   22]
 [   2    4    7    2   29]
 [   1    2  365  682    5]
 [   2    1  365    2    8]
 [   2    2   60   38   90]
 [   1    3   60  295   13]
 [   1    1 1095   19   17]
 [   1    1  730   19   11]
 [   1    2 1460  217   17]
 [   1    2   60   49   17]
 [   2    1  730   38    8]
 [   2    1   60    2   33]
 [   2    4   60    2   29]
 [   1    1  365   6

In [ ]:
#train = np.array(train)
#trainSet = train[:int(train.shape[0]*0.7), :]
#valSet = train[int(train.shape[0]*0.7):,:]

#labels = np.array(labels)
#trainLabelSet = labels[:int(labels.shape[0]*0.7)]
#valLabelSet = labels[int(labels.shape[0]*0.7):]


train = np.array(train)
trainSet = train[:70, :]
valSet = train[71:100,:]

labels = np.array(labels)
trainLabelSet = labels[:70]
valLabelSet = labels[71:100]

In [ ]:
import yaml

with open("./ToolBox/params.json") as data_file:
    #params = json.loads(data_file, object_hook=ascii_encode_dict)
    params = yaml.safe_load(data_file)
    
functions = CrossValidation.init(params, trainSet, trainLabelSet, valSet, valLabelSet)

best_config = functions["svm"](decision_function_shape="ovo")

best_config

Total configurations: 18
Starting configuration: 1, params: {'kernel': 'linear', 'C': 1}
Model fit finished. Calculating accuracy.
Current setting accuracy: 0.310344827586
Current best accuracy: 0.310344827586
------ Current setting execution time: 11 seconds = 0 minutes ------
Starting configuration: 2, params: {'kernel': 'linear', 'C': 10}
Model fit finished. Calculating accuracy.
Current setting accuracy: 0.275862068966
Current best accuracy: 0.310344827586
------ Current setting execution time: 49 seconds = 0 minutes ------
Starting configuration: 3, params: {'kernel': 'linear', 'C': 100}
Model fit finished. Calculating accuracy.
Current setting accuracy: 0.275862068966
Current best accuracy: 0.310344827586
------ Current setting execution time: 246 seconds = 4 minutes ------
Starting configuration: 4, params: {'kernel': 'poly', 'C': 1, 'degree': 3}


In [ ]:
from sklearn import svm

rf = svm.SVC(decision_function_shape='ovo')
rf.fit(train, labels)

In [ ]:
np.savetxt('./submission_SVM_OVO.csv', rf.predict(test_transformed), delimiter=',', fmt='%f')

In [ ]:
res = np.loadtxt('submission_SVM_OVO.csv', delimiter=',')

lis = []
for idx, y in enumerate(res):
    temp_lis = np.zeros(6)
    temp_lis[0] = idx+1
    temp_lis[y] = 1
    lis.append(temp_lis)

np.savetxt('submission_SVM_OVO.csv', lis, fmt='%d', delimiter=',')